# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df=pd.read_csv('Output Data/CityData.csv')
df

,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,pevek,69.7008,170.3133,0.57,90,100,3.36,RU,1659944642
1,1,zhigansk,66.7697,123.3711,16.06,63,100,5.20,RU,1659944643
2,2,hermanus,-34.4187,19.2345,13.78,75,40,1.11,ZA,1659944645
3,3,qaanaaq,77.4840,-69.3632,1.97,61,94,2.60,GL,1659944645
4,4,ushuaia,-54.8000,-68.3000,0.81,64,20,0.51,AR,1659944646
...,...,...,...,...,...,...,...,...,...,...
546,546,vieques,18.4250,-65.8329,28.01,78,40,5.66,PR,1659945225
547,547,vallenar,-28.5708,-70.7581,12.56,46,87,2.31,CL,1659945226
548,548,tartus,34.8890,35.8866,28.39,59,0,5.57,SY,1659945227
549,549,salinas,36.6777,-121.6555,17.29,79,0,3.09,US,1659945082


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations=df[['Lat','Lng']]
weights=df['Humidity']

In [4]:
fig=gmaps.figure(center=(20.0, 10.0),zoom_level=1.7)

fig.add_layer(gmaps.heatmap_layer(locations,weights,dissipating=False,max_intensity=100, point_radius=3))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
new_df=df.loc[(21<df['Max Temp'])&(df['Max Temp']<27)&(df['Wind Speed']<4.5)&(df['Cloudiness']==0)]

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df=new_df.dropna(how='any')

In [7]:
hotel_df['Hotel Name']=""
hotel_df

,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
55,55,gwanda,-20.9333,29.0000,25.52,15,0,3.28,ZW,1659944695,
58,58,kirovskiy,45.0917,133.5136,25.65,66,0,2.42,RU,1659944698,
61,61,junction city,39.0286,-96.8314,23.70,85,0,3.09,US,1659944200,
71,71,geneva,41.8875,-88.3054,24.92,93,0,2.57,US,1659944712,
136,136,jamestown,42.0970,-79.2353,23.77,73,0,3.60,US,1659944788,
199,199,sobreira,41.5237,-7.8023,26.53,48,0,1.37,PT,1659944857,
239,239,broome,42.2506,-75.8330,23.21,93,0,3.60,US,1659944905,
245,245,carroll,39.5501,-77.0164,25.18,89,0,1.79,US,1659944911,
258,258,bethel,41.3712,-73.4140,26.20,88,0,0.00,US,1659944923,
262,262,sesheke,-17.4759,24.2968,24.47,17,0,3.09,ZM,1659944927,


In [8]:
base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params={'radius':5000,'type':'hotel',"key": g_key,'keyword':'hotel'}

for index, row in hotel_df.iterrows():
    Lat=row['Lat']
    Lng=row['Lng']
    params['location']=f'{Lat},{Lng}'
    hotel=requests.get(base_url,params=params).json()
    
    try:
        hotel_df.loc[index,'Hotel Name']=hotel['results'][0]['name']
    except(KeyError,IndexError):
        hotel_df.loc[index,'Hotel Name']='na'
hotel_df

,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
55,55,gwanda,-20.9333,29.0000,25.52,15,0,3.28,ZW,1659944695,na
58,58,kirovskiy,45.0917,133.5136,25.65,66,0,2.42,RU,1659944698,"Uyut, Gostinitsa"
61,61,junction city,39.0286,-96.8314,23.70,85,0,3.09,US,1659944200,Courtyard by Marriott Junction City
71,71,geneva,41.8875,-88.3054,24.92,93,0,2.57,US,1659944712,Fairfield Inn & Suites by Marriott Chicago St....
136,136,jamestown,42.0970,-79.2353,23.77,73,0,3.60,US,1659944788,Chautauqua Harbor Hotel
199,199,sobreira,41.5237,-7.8023,26.53,48,0,1.37,PT,1659944857,Pena Park Hotel
239,239,broome,42.2506,-75.8330,23.21,93,0,3.60,US,1659944905,na
245,245,carroll,39.5501,-77.0164,25.18,89,0,1.79,US,1659944911,Best Western Westminster Hotel
258,258,bethel,41.3712,-73.4140,26.20,88,0,0.00,US,1659944923,La Quinta Inn & Suites by Wyndham Danbury
262,262,sesheke,-17.4759,24.2968,24.47,17,0,3.09,ZM,1659944927,Protea Hotel by Marriott Zambezi River Lodge


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
marker_layer=gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))